# U-Net Model

We'll be using a network architecture called "U-Net". The name of this network architecture comes from it's U-like shape when shown in a diagram like this (image from [U-net entry on wikipedia](https://en.wikipedia.org/wiki/U-Net)): 

<img src="u-net.jpg" width="600"/>

U-nets are commonly used for image segmentation. As you can see from the diagram, this architecture features a series of down-convolutions connected by max-pooling operations, followed by a series of up-convolutions connected by upsampling and concatenation operations. Each of the down-convolutions is also connected directly to the concatenation operations in the upsampling portion of the network. For more detail on the U-Net architecture, have a look at the original [U-Net paper by Ronneberger et al. 2015](https://arxiv.org/abs/1505.04597). 

We'll create a basic U-Net using Keras. 

**Basic Imports**

In [24]:
import keras
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Deconvolution3D
from keras.optimizers import Adam
from keras.layers.merge import concatenate
# Set the image shape to have the channels in the first dimension
K.set_image_data_format("channels_first")

**Depth of U-Net**

The "depth" of your U-Net is equal to the number of down-convolutions you will use. In the image above, the depth is 4 because there are 4 down-convolutions running down the left side including the very bottom of the U.

For the examples next, we'll use a U-Net depth of 2, meaning we'll have 2 down-convolutions in your network. 

The shape of the input layer is `(num_channels, height, width, length)`, where `num_channels` you can think of like color channels in an image, `height`, `width` and `length` are just the size of the input.

The values will be using will be:
- num_channels: 4
- height: 160
- width: 160
- length: 16

In [3]:
# Define an input layer tensor of the shape 
input_layer = Input(shape=(4, 160, 160, 16))
input_layer

W0428 16:52:19.885658 4442424768 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0428 16:52:19.912508 4442424768 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



<tf.Tensor 'input_1:0' shape=(?, 4, 160, 160, 16) dtype=float32>

Notice that the tensor shape has a '?' as the very first dimension.  This will be the batch size. So the dimensions of the tensor are: (batch_size, num_channels, height, width, length)

## Contracting (downward) path**

Here we'll be constructing the downward path in the network (the left size of the U-Net). The <mark>(height, width, length) </mark> of the input gets smaller as you move down this path, and the number of channels increases. 

**Depth 0**

By "depth 0" here, we are referring to the depth of the first down-convolution in the U-net. 

The number of filters is specified for each depth and for each layer within the depth. 

The formula to use for calculating the number of filters is:
$$\mathcal filters_{i} = 32 \times (2^{i}) $$

where $\mathcal i$ is the current depth. 

So at the depth $\mathcal i = 0$:
$$ \mathcal filters_{0} = 32 \times (2^{0}) = 32 $$

**Layer 0**

There are two convolutional layers for each depth. 

In [4]:
# Define a Conv3D tensor with 32 filters
down_depth_0_layer_0 = Conv3D(filters=32,
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1)
                             )(input_layer)
down_depth_0_layer_0

W0428 17:03:21.459342 4442424768 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0428 17:03:21.476487 4442424768 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0428 17:03:21.477242 4442424768 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:197: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



<tf.Tensor 'conv3d_1/add:0' shape=(?, 32, 160, 160, 16) dtype=float32>

Notice the 32 filters, the result you get above is a tensor with 32 channels.

In [5]:
# Add a relu activation to layer 0 of depth 0
down_depth_0_layer_0 = Activation('relu')(down_depth_0_layer_0)
down_depth_0_layer_0

<tf.Tensor 'activation_1/Relu:0' shape=(?, 32, 160, 160, 16) dtype=float32>

**Depth 0, Layer 1**

For layer 1 of depth 0, the formula you'll use for number of filters is:
$$ \mathcal_{i} = 32 \times (2^{i}) \times 2

Where $\mathcal i$ is the current depth. 

* Notice that the '$\times 2$' at the end of this expression isn't there for layer 0. 

So at depth $\mathcal i = 0$ for layer 1:
$$\mathcal filters_{0} = 32 \times (2^{0}) \times 2 = 64$$ 

In [7]:
# Create a Conv3D layer with 64 filters to your network and add relu activation. 
down_depth_0_layer_1 = Conv3D(filters=64, 
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1)
                             )(down_depth_0_layer_0)
down_depth_0_layer_1 = Activation('relu')(down_depth_0_layer_1)
down_depth_0_layer_1


<tf.Tensor 'activation_2/Relu:0' shape=(?, 64, 160, 160, 16) dtype=float32>

**Max Pooling**

Within the U-Net architecture, there is a max pooling operation after each of the down-convolutions (not including the last down-convolution at the bottom of the U). In general, this means you'll add max pooling after each down-convolution up to (but not including) the <mark>depth -1</mark> down convolution (since you started counting at 0).

In [8]:
# Define a max pooling layer
down_depth_0_layer_pool = MaxPooling3D(pool_size=(2,2,2))(down_depth_0_layer_1)
down_depth_0_layer_pool

<tf.Tensor 'max_pooling3d_1/transpose_1:0' shape=(?, 64, 80, 80, 8) dtype=float32>

**Depth 1, Layer 0**

At depth 1, layer 0, the formula for calculating the number of filters is:
$$\mathcal filters_{i} = 32 \times (2^{i})$$

Where $\mathcal i$ is the current depth.

So at depth $\mathcal i = 1$:
$$\mathcal filters_{i} = 32 \times (2^{1}) = 64$$

The next cell adds a Conv3D layer to the network with relu activation. 

In [10]:
# Add a Conv3D layer to your network with relu activation
down_depth_1_layer_0 = Conv3D(filters=64, 
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1)
                             )(down_depth_0_layer_pool)
down_depth_1_layer_0 = Activation('relu')(down_depth_1_layer_0)
down_depth_1_layer_0

<tf.Tensor 'activation_3/Relu:0' shape=(?, 64, 80, 80, 8) dtype=float32>

**Depth 1, Layer 1**

For layer 1 of depth 1 the formula you'll use for number of filters is:

$$\mathcal filters_{i} = 32 \times (2^{i}) \times 2$$

Where $\mathcal i$ is the current depth.

* Notice that the $'\times 2'$ at the end of this expression isn't there for layer 0. 

So at depth $\mathcal = 1$:

$$ \mathcal filters_{0} = 32 \times (2^{1}) \times 2 = 128 $$

The next cell adds another Conv3D with 128 filters to the network. 

In [12]:
# Add another Conv3D with 128 filters to your network. 
down_depth_1_layer_1 = Conv3D(filters=128, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(down_depth_1_layer_0)
down_depth_1_layer_1 = Activation('relu')(down_depth_1_layer_1)
down_depth_1_layer_1

<tf.Tensor 'activation_5/Relu:0' shape=(?, 128, 80, 80, 8) dtype=float32>

**No max pooling at depth 1(the bottom of the U)**

At the "bottom" of the U-net, there is no need to apply max pooling after the convolutions.  

## Expanding (upward) Path

Now we'll work on the expanding path of the U-Net, (going up on the right side, when viewing the diagram). The image's (height, width, length) all get larger in the expanding path.

**Depth 0, Up sampling layer 0**

We'll be using a pool size of (2,2,2) for upsampling. 

- This is the default value for [tf.keras.layers.UpSampling3D]
(https://www.tensorflow.org/api_docs/python/tf/keras/layers/UpSampling3D)
- As input to the upsampling at depth 1, you'll use the last layer of the downsampling. In this case, it's a depth 1 layer 1. 

Next cell adds an upsampling operation to your network. Note that we are not adding any activation to this upsampling layer.  

In [13]:
# Add an upsampling operation to your network
up_depth_0_layer_0 = UpSampling3D(size=(2,2,2))(down_depth_1_layer_1)
up_depth_0_layer_0

<tf.Tensor 'up_sampling3d_1/concat_2:0' shape=(?, 128, 160, 160, 16) dtype=float32>

**Concatenate upsampled depth 0 with downsampled depth 0**

Now you'll apply a concatenation operation using the layers that are both at the same depth of 0. 

* up_depth_0_layer_0: shape is (?, 128, 160, 160, 16)
* depth_0_layer_1: shape is (?, 64, 160, 160, 16)
* Double check that both these layers have the same height, width and length. 
* The (height, width, length) is (160, 160, 16) for both. 

The next cell checks that the layers we wish to concatenate have the same height, width and length. 

In [14]:
# Print the shape of layers to concatenate
print(up_depth_0_layer_0)
print()
print(down_depth_0_layer_1)

Tensor("up_sampling3d_1/concat_2:0", shape=(?, 128, 160, 160, 16), dtype=float32)

Tensor("activation_2/Relu:0", shape=(?, 64, 160, 160, 16), dtype=float32)


The next cell will add a concatenation operation to your network. 

In [15]:
# Add a concate along axis 1
up_depth_1_concat = concatenate([up_depth_0_layer_0,
                                 down_depth_0_layer_1],
                                 axis=1)
up_depth_1_concat

<tf.Tensor 'concatenate_1/concat:0' shape=(?, 192, 160, 160, 16) dtype=float32>

Notice that the upsampling layer had 128 channels, and the down-convolution layer had 64 channels so that when concatenated, the results has 128 + 64 = 192 channels. 

**Up-convolution layer 1**

The number of filters for this layer will be set to the number of channels is the down-convolution's layer 1 at the same depth of 0 (down_depth_0_layer_1). 

The next cell displays the shape of the down-convolution depth 0 layer 1. 

In [16]:
down_depth_0_layer_1

<tf.Tensor 'activation_2/Relu:0' shape=(?, 64, 160, 160, 16) dtype=float32>

Notice that the number of channels for <mark>depth_0_layer_1</mark> is 64. 

In [26]:
print(f"number of filters: {down_depth_0_layer_1._keras_shape[1]}")

number of filters: 64


In [27]:
# Add a Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_1 = Conv3D(filters=64,
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_concat)
up_depth_1_layer_1 = Activation('relu')(up_depth_1_layer_1)
up_depth_1_layer_1

<tf.Tensor 'activation_6/Relu:0' shape=(?, 64, 160, 160, 16) dtype=float32>

**Up-convolution depth 0, layer 2**

At layer 2 of depth 0 in the up-convolution the next step will be to add another up-convolution. The number of filters you'll want to use for this next up-convolution will need to be equal to the number of filters in the down-convolution depth 0 layer 1. 

The next cell reminds of the number of filters in down-convolution depth 0  layer 1. 

In [28]:
print(down_depth_0_layer_1)
print(f'number of filters: {down_depth_0_layer_1._keras_shape[1]}')

Tensor("activation_2/Relu:0", shape=(?, 64, 160, 160, 16), dtype=float32)
number of filters: 64


As you can see, the number of channels/filters in <mark>down_depth_0_layer_1</mark> is 64. 

The next cell to add a Conv3D up-convolution with 64 filters to your network. 

In [29]:
# Add a Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_2 = Conv3D(filters=64,
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_layer_1)
up_depth_1_layer_2 = Activation('relu')(up_depth_1_layer_2)
up_depth_1_layer_2

<tf.Tensor 'activation_7/Relu:0' shape=(?, 64, 160, 160, 16) dtype=float32>

**Final Convolution**

For the final convolution, you'll set the number of filters to be equal to the number of classes in your input data. 

Here, we have 3 classes, namely:

* 1:edema
* 2:non-enhancing tumor
* 3:enhancing tumor

The next cell adds a final Conv3D with 3 filters to your network

In [30]:
# Add a final Conv3D with 3 filters to your network. 
final_conv = Conv3D(filters=3, #3 categories
                    kernel_size=(1,1,1),
                    padding='valid',
                    strides=(1,1,1)
                   )(up_depth_1_layer_2)
final_conv

<tf.Tensor 'conv3d_10/add:0' shape=(?, 3, 160, 160, 16) dtype=float32>

**Activation for final convolution**

The next cell adds a sigmoid activation function to your final convolution. 

In [31]:
# Add a sigmoid activation to your final convolution. 
final_activation = Activation('sigmoid')(final_conv)
final_activation

<tf.Tensor 'activation_8/Sigmoid:0' shape=(?, 3, 160, 160, 16) dtype=float32>

**Create and compile the model**

In [33]:
# Define and compile your model
model = Model(inputs=input_layer, outputs=final_activation)
model.compile(optimizer=Adam(lr=0.00001),
              loss='categorical_crossentropy', 
              metrics=['categorical_accuracy']
             )

W0429 13:26:25.004676 4442424768 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [35]:
# Print out a summary of the model we have created
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 4, 160, 160,  0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 32, 160, 160, 3488        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 160, 160, 0           conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_3 (Conv3D)               (None, 64, 160, 160, 55360       activation_1[0][0]               
____________________________________________________________________________________________

**Double check your model**

To double check that we've created the correct model, use a function that we've provided to create the same model, and check that the layers and layer dimensions match!

In [39]:
# Import predefined utilities
import util_3

In [41]:
# Create a model using a predefined function
model_2 = util_3.unet_model_3d(depth=2,
                             loss_function='categorical_crossentropy',
                             metrics=['categorical_accuracy'])

In [42]:
# Print out a summary of the model created by the predefined function 
model_2.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 4, 160, 160,  0                                            
__________________________________________________________________________________________________
conv3d_11 (Conv3D)              (None, 32, 160, 160, 3488        input_2[0][0]                    
__________________________________________________________________________________________________
activation_9 (Activation)       (None, 32, 160, 160, 0           conv3d_11[0][0]                  
__________________________________________________________________________________________________
conv3d_12 (Conv3D)              (None, 64, 160, 160, 55360       activation_9[0][0]               
____________________________________________________________________________________________